# Микродиахроническое исследование русских приставок методами дистрибутивной семантики
## Автор: Елизавета Клыкова, БКЛ181
### Подготовка данных
1. Считать предложения из файлов
2. Удалить дубли
3. Удалить мусор (пустые предложения, предложения из пунктуации, предложения не на русском)
4. Очистить от лишних символов слева
5. Лемматизировать и получить частеречные теги (Mystem)
6. Сохранить все в файлы .json

#### Импорт модулей

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import re
import os
import json
import math
import jsonlines
from tqdm.auto import tqdm
from string import punctuation

#### Открываем и считываем файлы

In [3]:
with open('rnc_pre-soviet.txt', encoding='utf-8') as f1:
    pre_soviet = [line.strip() for line in f1.readlines()]

len(pre_soviet)  # длина в предложениях

5247428

In [4]:
presov = list(set(pre_soviet))
len(presov)  # число уникальных предложений

4457762

#### Чистим датасеты

In [5]:
punct = punctuation + '«»…– '

In [6]:
def clean_sentences(sentences):
    clean_sents = []
    for sent in tqdm(sentences):
        # убираем переносы строки
        sent = sent.replace('\n', '')
        # убираем пустые и мусорные предложения
        if re.search(r'[а-яА-ЯёЁ]', sent):
            # обрезаем пунктуацию с левого края
            clean_sents.append(sent.lstrip(punct))
    return clean_sents

In [7]:
clean_presov = clean_sentences(presov)
len(clean_presov)

  0%|          | 0/4457762 [00:00<?, ?it/s]

4448941

#### Лемматизируем

In [8]:
def parse_with_mystem(input_fname, output_fname, sentences):

    with open(input_fname, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sentences))

    # -с -- полнотекстовое представление
    # -i -- грамматическая информация
    # -g -- склеивание грамматической информации
    # -d -- снятие омонимии
    os.system(
        r'.\mystem.exe -cigd --format json {} {}'.format(
            input_fname, output_fname))

In [9]:
def parse_batches(input_prefix, output_prefix, sentences,
                  batch_len=500000):

    num_batches = math.ceil(len(sentences) / batch_len)

    for i in tqdm(range(num_batches)):

        print(f'Processing batch {i}...')
        start_index = i * batch_len
        end_index = (i + 1) * batch_len
        if end_index > len(sentences):
            end_index = len(sentences)

        # лемматизация
        parse_with_mystem(f'{input_prefix}{i}.txt',
                          f'{output_prefix}{i}.json',
                          sentences[start_index:end_index])
        print()

In [10]:
parse_batches('presov', 'presov_lem', clean_presov)

  0%|          | 0/9 [00:00<?, ?it/s]

Processing batch 0...

Processing batch 1...

Processing batch 2...

Processing batch 3...

Processing batch 4...

Processing batch 5...

Processing batch 6...

Processing batch 7...

Processing batch 8...



#### Освобождаем память и повторяем для оставшихся двух периодов

In [11]:
del clean_presov

**Советский**

In [12]:
with open('rnc_soviet.txt', encoding='utf-8') as f2:
    soviet = [line.strip() for line in f2.readlines()]

len(soviet)

8068459

In [13]:
sov = list(set(soviet))
len(sov)

7040411

In [14]:
clean_sov = clean_sentences(sov)
len(clean_sov)

  0%|          | 0/7040411 [00:00<?, ?it/s]

7033181

In [15]:
parse_batches('sov', 'sov_lem', clean_sov)

  0%|          | 0/15 [00:00<?, ?it/s]

Processing batch 0...

Processing batch 1...

Processing batch 2...

Processing batch 3...

Processing batch 4...

Processing batch 5...

Processing batch 6...

Processing batch 7...

Processing batch 8...

Processing batch 9...

Processing batch 10...

Processing batch 11...

Processing batch 12...

Processing batch 13...

Processing batch 14...



In [16]:
del clean_sov

**Пост-советский**

In [17]:
with open('rnc_post-soviet.txt', encoding='utf-8') as f3:
    post_soviet = [line.strip() for line in f3.readlines()]

len(post_soviet)

6569778

In [18]:
postsov = list(set(post_soviet))
len(postsov)

5878273

In [19]:
clean_postsov = clean_sentences(postsov)
len(clean_postsov)

  0%|          | 0/5878273 [00:00<?, ?it/s]

5853824

In [20]:
parse_batches('postsov', 'postsov_lem', clean_postsov)

  0%|          | 0/12 [00:00<?, ?it/s]

Processing batch 0...

Processing batch 1...

Processing batch 2...

Processing batch 3...

Processing batch 4...

Processing batch 5...

Processing batch 6...

Processing batch 7...

Processing batch 8...

Processing batch 9...

Processing batch 10...

Processing batch 11...



In [21]:
del clean_postsov

#### Делим файлы, т.к. они оказались слишком большими

In [ ]:
presov_files, sov_files, postsov_files = [], [], []

for fname in os.listdir():
    if fname.startswith('presov_lem'):
        presov_files.append(fname)
    elif fname.startswith('sov_lem'):
        sov_files.append(fname)
    elif fname.startswith('postsov_lem'):
        postsov_files.append(fname)

In [ ]:
all_files = presov_files + sov_files + postsov_files

In [ ]:
def read_mystem_results(filename):
    print('Loading file {}...'.format(filename))
    with open(filename, encoding='utf-8') as f:
        lines = []
        for line in f.readlines():
            lines.append(json.loads(line))
    print('File {} loaded successfully!'.format(filename))
    return lines

In [ ]:
for filename in all_files:
    lines = read_mystem_results(filename)
    newfile1 = filename.split('.')[0] + '_1' + '.json'
    newfile2 = filename.split('.')[0] + '_2' + '.json'
    with jsonlines.open(newfile1, 'w') as writer:
        writer.write_all(lines[0:250000])
    with jsonlines.open(newfile2, 'w') as writer:
        writer.write_all(lines[250000:])
    del lines